In [2]:
!pip install requests_html

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 5.3 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=e045f3247d26eee8be587fc4d42adb19a56d974cf6e213c0f2ff3872a0524678
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from requests_html import HTMLSession
import pandas as pd
import numpy
import urllib.robotparser
import time
from bs4 import BeautifulSoup
import json

In [5]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.cars-data.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.cars-data.com/en/all-cars.html')

False

In [6]:
rp.can_fetch('*', 'https://www.cars-data.com/en/audi-rs4-avant-2.9-tfsi-quattro-specs/80342')

False

In [7]:
url = 'https://www.cars-data.com/en/all-cars.html'
session = HTMLSession()

In [8]:
res = session.get(url)

In [9]:
car_urls = set()

In [10]:
# function craw links info car in pages
def crawl_link_cars(cars_in_page):
    for car in cars_in_page:
        url = car.find('a', first = True).attrs['href']
        car_urls.add(url)

In [11]:
for html in res.html:
    cars_in_page = html.find('section.models > div.col-4')
    print(html.url)
    print(len(cars_in_page))
    crawl_link_cars(cars_in_page)
    print()

https://www.cars-data.com/en/all-cars.html
48

https://www.cars-data.com/en/all-cars/page2.html
48

https://www.cars-data.com/en/all-cars/page3.html
48

https://www.cars-data.com/en/all-cars/page4.html
48

https://www.cars-data.com/en/all-cars/page5.html
48

https://www.cars-data.com/en/all-cars/page6.html
48

https://www.cars-data.com/en/all-cars/page7.html
48

https://www.cars-data.com/en/all-cars/page8.html
48

https://www.cars-data.com/en/all-cars/page9.html
48

https://www.cars-data.com/en/all-cars/page10.html
48

https://www.cars-data.com/en/all-cars/page11.html
48

https://www.cars-data.com/en/all-cars/page12.html
48

https://www.cars-data.com/en/all-cars/page13.html
48

https://www.cars-data.com/en/all-cars/page14.html
48

https://www.cars-data.com/en/all-cars/page15.html
48

https://www.cars-data.com/en/all-cars/page16.html
48

https://www.cars-data.com/en/all-cars/page17.html
48

https://www.cars-data.com/en/all-cars/page18.html
48

https://www.cars-data.com/en/all-cars/page1

In [12]:
list_car_urls = list(car_urls)

In [13]:
len(list_car_urls)

4637

In [14]:
df = pd.DataFrame({'urls': list_car_urls})
df.to_csv("/content/drive/MyDrive/Data/car_urls.csv", sep = '\t')

In [15]:
cars_df = pd.read_csv('/content/drive/MyDrive/Data/car_urls.csv', sep = '\t')

In [16]:
cars = cars_df['urls']
cars.head()

0    https://www.cars-data.com/en/porsche-911-carre...
1    https://www.cars-data.com/en/toyota-celica-1-6...
2    https://www.cars-data.com/en/volkswagen-jetta-...
3    https://www.cars-data.com/en/mitsubishi-lancer...
4    https://www.cars-data.com/en/mercedes-benz-sl-...
Name: urls, dtype: object

In [17]:
s = HTMLSession()

In [18]:
def get_car_info(car):
    r = s.get(car)
    # Get
    dts = r.html.find('td.col-6')
    dds = r.html.find('dd.col-6')
    price = 0.0
    eLabel = 'NULL'
    length = 0.0

    for i in range(len(dts)):
        if (dts[i].text == 'Price:'):
            print(dts[i+1].text)
            price = dts[i+1].text.split()[-1]
        if (dts[i].text == 'energy label:'):
            eLabel = dts[i+1].text
        if (dts[i].text == 'length:'):
            length = dts[i+1].text.split()[0]

    soup = BeautifulSoup(r.html.html, 'html.parser')
    js = json.loads(soup.select_one("script[type='application/ld+json']").text)
    name = js.get('name', "NA")
    bodyType = js.get('bodyType', "NA")
    emissionsCO2 = js.get('emissionsCO2', "NA")
    modelDate = js.get('modelDate', "NA")
    fuelType = js.get('fuelType', "NA")
    numberOfAxles = js.get('numberOfAxles', "NA")
    numberOfDoors = js.get('numberOfDoors', "NA")
    numberOfForwardGears = js.get('numberOfForwardGears', "NA")
    seatingCapacity = js.get('seatingCapacity', "NA")
    vehicleTransmission = js.get('vehicleTransmission', "NA")
    model = js.get('model', "NA")
    url  = js.get('mainEntityOfPage', "NA")
    cargoVolume = js.get('cargoVolume', "NA").get('value', "NA")
    roofLoad = js.get('roofLoad', "NA").get('value', "NA")
    accelerationTime = js.get('accelerationTime', "NA").get('value', "NA")
    fuelCapacity = js.get('fuelCapacity', "NA").get('value', "NA")
    fuelConsumption = js.get('fuelConsumption', "NA").get('value', "NA")
    speed = js.get('speed', "NA").get('value', "NA")
    payload = js.get('payload', "NA").get('value', "NA")
    trailerWeight = js.get('trailerWeight', "NA").get('value', "NA")
    vEengineType = js.get('vehicleEngine', {})[0].get('engineType', "NA")
    vEfuelType = js.get('vehicleEngine', {})[0].get('fuelType', "NA")
    vEengineDisplacement = js.get('vehicleEngine', {})[0].get('engineDisplacement', "NA").get('value', "NA")
    vEenginePower = js.get('vehicleEngine', {})[0].get('enginePower', "NA").get('value', "NA")
    torque = js.get('vehicleEngine', {})[0].get('torque', "NA").get('value', "NA")
    weightTotal = js.get('weightTotal', "NA").get('value', "NA")
    wheelbase = js.get('wheelbase', "NA")
    height = js.get('height', "NA").get('value', "NA")
    brand = js.get('manufacturer', {})
    weight = js.get('weight', "NA").get('value', "NA")
    width = js.get('width', "NA").get('value', "NA")
    # print(vEengineDisplacement)
    # print(soup)
    # print(type(vEenginePower),type(vEengineType),type(vEfuelType),type(vEengineDisplacement),type(torque),type(numberOfForwardGears),type(seatingCapacity),type(height),type(width),type(weight),type(weightTotal))
    values = url + '\t' + name + '\t' + model + '\t' + brand + '\t' + str(price) + '\t' + eLabel + '\t' + bodyType + '\t' + str(length) + '\t' + height + '\t' + width + '\t' + weight + '\t' + weightTotal + '\t' + emissionsCO2 + '\t' + modelDate + '\t' + fuelType + '\t' + numberOfAxles + '\t' + numberOfDoors + '\t' + numberOfForwardGears + '\t' + seatingCapacity + '\t' + vehicleTransmission + '\t' + cargoVolume + '\t' + roofLoad + '\t' + accelerationTime + '\t' + '\t' + fuelCapacity + '\t' + fuelConsumption + '\t' + speed + '\t' + payload + '\t' + trailerWeight + '\t' + vEengineType + '\t' + vEfuelType + '\t' + vEengineDisplacement + '\t' + vEenginePower + '\t' + torque

    return values

In [19]:
def get_partOf_Cars_data(s, e):
    count = 0
    fileName = '/content/drive/MyDrive/Data/cars_' + str(s) + '_to_' + str(e) +'.csv'
    f = open(fileName, 'w+', encoding='utf-8')
    start = time.time()
    for car in cars[s:e]:
        print('.', end='')
        count += 1
        values = get_car_info(car) + '\n'
        f.write(values)
        if (count == 100):
            print()
            count = 0
    print()
    print('%0.2f' %(time.time() - start))
    f.close()

In [20]:
def get_Cars_url(url):
    fileName = '/content/drive/MyDrive/Data/cars_info' + '.csv'
    f = open(fileName, 'a', encoding='utf-8')
    values = get_car_info(url) + '\n'
    f.write(values)
    f.close()

In [21]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed

In [22]:
def craw_mutilthread(urls,max_worker = 4):
    with ThreadPoolExecutor(max_workers = max_worker) as executor:
      executor.map(get_Cars_url, urls, timeout = 60)


In [23]:
craw_mutilthread(cars[0:20])

€ 22.117
€ 97.971
€ 8.145
€ 18.999
€ 135.068
€ 7.950
€ 63.344
€ 6.260
€ 96.565
€ 72.378
€ 19.340
€ 25.995
€ 27.225
€ 19.490
€ 55.711
€ 8.892
€ 36.711
€ 5.125
€ 27.365
€ 13.611


In [ ]:
get_partOf_Cars_data(1,10)

.€ 19.717
.€ 9.073
.€ 76.975
.€ 65.083
.€ 24.145
.€ 26.773
.€ 47.435
.€ 37.391
.€ 7.258

2.31


In [ ]:
get_partOf_Cars_data(0,10)

.€ 23.400
.€ 19.717
.€ 9.073
.€ 76.975
.€ 65.083
.€ 24.145
.€ 26.773
.€ 47.435
.€ 37.391
.€ 7.258

4.17
